In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [40]:
#Datei einlesen und anzeigen:
df = pd.read_csv("/Users/carareusch/Desktop/TechsLabTeam2/TechsLabTeam2-3/Data/Raw/Hotel/hotel_bookings.csv", encoding="ISO-8859-1", sep=',')

pd.set_option('display.max_columns', None)

#Neue Spalten für Gesamtaufenthaltsdauer und Gesamtzahl der Gäste:
df["duration_of_stay"]= df["stays_in_weekend_nights"] + df["stays_in_week_nights"]
df["total_guests"]= df["adults"] + df["children"] + df["babies"]


#Wie viele Buchungen gibt es, bei denen 0 Tage als Gesamtdauer angegeben sind:
print(f"Es gibt insgesamt {df["duration_of_stay"].value_counts().get (0)} Buchungen mit 0 Tagen, die für die nächsten Berechechnungen raus gerechnet werden.")

#Anpassung des Dataframes, sodass alle Buchungen mit 0 Tagen raufallen
df= df[df["duration_of_stay"]!=0]

#Weitere Untersuchungen:
print (f"Es bleiben dann immer noch {df["total_guests"].value_counts().get(0)} Buchungen übrig, in denen kein Gast kommmt.")
print (f"Es gibt außerdem {df["adults"].value_counts().get(0)} Buchungen ohne Erwachsene")


df_children_alone=df[df["adults"]==0]
df_children_alone["children"].value_counts()
print(f"Folgende Kinder kämen ohne Erwachsenen: {df_children_alone["children"].value_counts().get(1)} x 1 Kind, {df_children_alone["children"].value_counts().get(2)} x 2 Kinder, {df_children_alone["children"].value_counts().get(3)} x 3 Kinder.")
print(f"Es gibt {df_children_alone["babies"].value_counts().get(1)} x ein Baby, dass ohne Erwachsene kommt")




Es gibt insgesamt 715 Buchungen mit 0 Tagen, die für die nächsten Berechechnungen raus gerechnet werden.
Es bleiben dann immer noch 110 Buchungen übrig, in denen kein Gast kommmt.
Es gibt außerdem 333 Buchungen ohne Erwachsene
Folgende Kinder kämen ohne Erwachsenen: 4 x 1 Kind, 208 x 2 Kinder, 11 x 3 Kinder.
Es gibt 3 x ein Baby, dass ohne Erwachsene kommt


In [41]:
#Wie verhalten sich die Buchungen ohne Kinder?

#Dataframe anpassen, damit alle Buchungen mit Kindern oder babies wegfallen:
df_without_children = df[(df['children'] == 0) & (df['babies'] == 0) & (df["adults"] != 0)] #zusätzlich alle Buchungen rausnehmen, in denen adults null sind, da haben wir offenbar keine Werte

# DataFrame nach der Spalte "Hotel" gruppieren
hotel_grp_adults = df_without_children.groupby("hotel")

#Wie viele Buchungen mit nur einer Person (Dienstreise?) gibt es im Hotel?
# Zählen, wie oft die Zahl 1 in der Spalte "adults" für jedes Hotel separat vorkommt
adults_count_resort_one = hotel_grp_adults.get_group("Resort Hotel")["adults"].value_counts().get(1, 0)
adults_count_city_one = hotel_grp_adults.get_group("City Hotel")["adults"].value_counts().get(1, 0)

# Ausgabe der Ergebnisse
print("Anzahl der Buchungen mit nur einer Person im Resort Hotel:", adults_count_resort_one)
print("Anzahl der Buchungen mit nur einer Person mit im City Hotel:", adults_count_city_one)

#Wie verhalten sich die Buchungen in Bezug auf die Anzahl von Erwachsenen Personen insgesamt?
adults_count_resort = hotel_grp_adults.get_group("Resort Hotel")["adults"].value_counts()
adults_count_city = hotel_grp_adults.get_group("City Hotel")["adults"].value_counts()

# Ausgabe der Ergebnisse
print("Resort Hotel:", adults_count_resort)
print(" City Hotel:", adults_count_city)

Anzahl der Buchungen mit nur einer Person im Resort Hotel: 6905
Anzahl der Buchungen mit nur einer Person mit im City Hotel: 15472
Resort Hotel: adults
2     27793
1      6905
3      1020
4        28
26        5
27        2
20        2
5         2
40        1
50        1
55        1
6         1
10        1
Name: count, dtype: int64
 City Hotel: adults
2    53371
1    15472
3     4632
4       31
Name: count, dtype: int64


In [42]:
# Liste der ungewöhnlichen Werte für Erwachsene im Resort Hotel
unusual_values_resort = [26, 27, 20, 40, 55, 10, 50]

# Extrahiere die Zeilen mit ungewöhnlichen Werten für Erwachsene im Resort Hotel
unusual_bookings_resort = df_without_children[(df_without_children['hotel'] == 'Resort Hotel') & (df_without_children['adults'].isin(unusual_values_resort))]

# Zeige die extrahierten Zeilen
display(unusual_bookings_resort)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,duration_of_stay,total_guests
1539,Resort Hotel,1,304,2015,September,36,3,0,3,40,0.0,0,BB,PRT,Direct,Direct,0,0,0,A,A,0,No Deposit,NaN,NaN,0,Group,0.0,0,0,Canceled,2015-01-02,3,40.0
1587,Resort Hotel,1,333,2015,September,36,5,2,5,26,0.0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,96.0,NaN,0,Group,0.0,0,0,Canceled,2015-01-02,7,26.0
1643,Resort Hotel,1,336,2015,September,37,7,1,2,50,0.0,0,BB,PRT,Direct,Direct,0,0,0,A,A,0,No Deposit,NaN,NaN,0,Group,0.0,0,0,Canceled,2015-01-18,3,50.0
1752,Resort Hotel,1,340,2015,September,37,12,2,5,26,0.0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,96.0,NaN,0,Group,0.0,0,0,Canceled,2015-01-02,7,26.0
1884,Resort Hotel,1,347,2015,September,38,19,2,5,26,0.0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,96.0,NaN,0,Group,0.0,0,0,Canceled,2015-01-02,7,26.0
1917,Resort Hotel,1,349,2015,September,39,21,1,3,27,0.0,0,HB,PRT,Direct,Direct,0,0,0,A,A,0,No Deposit,NaN,NaN,0,Group,0.0,0,0,Canceled,2015-01-02,4,27.0
1962,Resort Hotel,1,352,2015,September,39,24,1,3,27,0.0,0,HB,PRT,Direct,Direct,0,0,0,A,A,0,No Deposit,NaN,NaN,0,Group,0.0,0,0,Canceled,2015-01-02,4,27.0
2003,Resort Hotel,1,354,2015,September,39,26,2,5,26,0.0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,96.0,NaN,0,Group,0.0,0,0,Canceled,2015-01-02,7,26.0
2164,Resort Hotel,1,361,2015,October,40,3,2,5,26,0.0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,96.0,NaN,0,Group,0.0,0,0,Canceled,2015-01-02,7,26.0
2173,Resort Hotel,1,338,2015,October,41,4,2,0,55,0.0,0,HB,PRT,Direct,Direct,0,0,0,A,A,0,No Deposit,NaN,NaN,0,Group,0.0,0,0,Canceled,2015-01-02,2,55.0


In [43]:
#Wie verändern sich die Werte oben, wenn man die Cancellation nochmal separat rausrechnet:

#Anpassung des Dataframes, sodass alle Buchungen, die gecancelt wurden raufallen
df= df[df["is_canceled"]==0]

#Wie viele Buchungen gibt es, bei denen 0 Tage als Gesamtdauer angegeben sind:
print(f"Es gibt insgesamt {df["duration_of_stay"].value_counts().get (0)} Buchungen mit 0 Tagen, die für die nächsten Berechechnungen raus gerechnet werden.")

#Anpassung des Dataframes, sodass alle Buchungen mit 0 Tagen raufallen
#df= df[df["duration_of_stay"]!=0]

#Weitere Untersuchungen:
print (f"Es bleiben dann immer noch {df["total_guests"].value_counts().get(0)} Buchungen übrig, in denen kein Gast kommmt.")
print (f"Es gibt außerdem {df["adults"].value_counts().get(0)} Buchungen ohne Erwachsene")


df_children_alone=df[df["adults"]==0]
df_children_alone["children"].value_counts()
print(f"Folgende Kinder kämen ohne Erwachsenen: {df_children_alone["children"].value_counts().get(1)} x 1 Kind, {df_children_alone["children"].value_counts().get(2)} x 2 Kinder, {df_children_alone["children"].value_counts().get(3)} x 3 Kinder.")
print(f"Es gibt {df_children_alone["babies"].value_counts().get(1)} x ein Baby, dass ohne Erwachsene kommt")


Es gibt insgesamt None Buchungen mit 0 Tagen, die für die nächsten Berechechnungen raus gerechnet werden.
Es bleiben dann immer noch 97 Buchungen übrig, in denen kein Gast kommmt.
Es gibt außerdem 236 Buchungen ohne Erwachsene
Folgende Kinder kämen ohne Erwachsenen: 4 x 1 Kind, 127 x 2 Kinder, 8 x 3 Kinder.
Es gibt 2 x ein Baby, dass ohne Erwachsene kommt


In [47]:
# Filtern nach den Bedingungen
filtered_df = df_children_alone[(df_children_alone["children"].isin([1, 2, 3])) | (df_children_alone["babies"] == 1)]

print("Alle Kinder kommen nur alleine ins City Hotel. Die Babies reisen immer mit anderen Kindern.")
#Vielleicht wurde für die allein Reisenden Kindern, einfach ein extra Zimmer gebucht? Bei Customer_type steht: 
#Transient-Party, das bedeutet laut Erklärung des Datasets: Transient-party – when the booking is transient, but is associated to at least other transient booking

# Setze die Anzeigeoption für die maximale Anzahl von Zeilen auf None
pd.set_option('display.max_rows', None)

# Anzeigen der gefilterten Zeilen
display(filtered_df)


Alle Kinder kommen nur alleine ins City Hotel. Die Babies reisen immer mit anderen Kindern.


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,duration_of_stay,total_guests
40984,City Hotel,0,1,2015,August,33,10,1,1,0,3.0,0,BB,PRT,Direct,Direct,0,0,0,B,B,1,No Deposit,NaN,NaN,0,Transient-Party,9.00,0,0,Check-Out,2015-08-12,2,3.0
41048,City Hotel,0,104,2015,August,33,11,0,3,0,2.0,0,BB,FRA,Online TA,TA/TO,0,0,0,B,B,1,No Deposit,7.0,NaN,0,Transient-Party,6.00,0,2,Check-Out,2015-08-14,3,2.0
41446,City Hotel,0,3,2015,August,34,16,2,0,0,2.0,0,BB,PRT,Direct,Direct,0,0,0,B,B,1,No Deposit,NaN,NaN,0,Transient-Party,6.00,0,1,Check-Out,2015-08-18,2,2.0
41952,City Hotel,0,15,2015,August,35,28,0,1,0,2.0,0,BB,PRT,Complementary,Direct,0,0,0,B,B,0,No Deposit,NaN,45.0,0,Transient,0.00,0,1,Check-Out,2015-08-29,1,2.0
46150,City Hotel,0,6,2015,December,51,18,0,1,0,2.0,1,BB,PRT,Online TA,TA/TO,0,0,0,B,B,0,No Deposit,9.0,NaN,0,Transient-Party,77.00,0,2,Check-Out,2015-12-19,1,3.0
46186,City Hotel,0,1,2015,December,52,23,0,3,0,2.0,0,BB,DEU,Online TA,TA/TO,0,0,0,B,B,0,No Deposit,9.0,NaN,0,Transient-Party,6.00,0,0,Check-Out,2015-12-26,3,2.0
46220,City Hotel,0,7,2015,December,52,26,2,1,0,2.0,0,BB,ESP,Online TA,TA/TO,0,0,0,B,B,0,No Deposit,9.0,NaN,0,Transient-Party,83.58,0,0,Check-Out,2015-12-29,3,2.0
46488,City Hotel,0,30,2016,January,2,3,1,0,0,2.0,0,BB,PRT,Online TA,TA/TO,0,0,0,B,B,2,No Deposit,9.0,NaN,0,Transient-Party,15.00,0,1,Check-Out,2016-01-04,1,2.0
46546,City Hotel,0,33,2016,January,2,6,0,2,0,2.0,0,BB,BRA,Online TA,TA/TO,0,0,0,B,B,0,No Deposit,9.0,NaN,0,Transient-Party,6.00,0,1,Check-Out,2016-01-08,2,2.0
46559,City Hotel,0,0,2016,January,2,7,0,1,0,2.0,0,BB,FRA,Online TA,TA/TO,0,0,0,B,B,0,No Deposit,9.0,NaN,0,Transient-Party,6.00,0,0,Check-Out,2016-01-08,1,2.0
